In [1]:
from tensorflow.keras import optimizers, layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
DB_PATH = r"dataset.csv"
TEST_SIZE = .2
VALIDATION_SIZE = .15
BATCH_SIZE = 32
EPOCHS = 100
DTYPE = 'float32'
LEARNING_RATE = 2e-3
RANDOM_STATE = 123
DROP_COLS = ['id']  # , 'dataset'
CAT_COLS = ['sex', 'cp', 'restecg', 'slope', 'thal']
SCALE_COLS = ['trestbps', 'chol', 'thalch']
RENAME_COLS = {
    'num': 'target'
}

In [3]:
class MLP(Model):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.activation = layers.LeakyReLU()
        self.dropout = layers.Dropout(.15)
        self.classifier = layers.Dense(output_dim, activation='softmax')
        self.hidden_layers = [
            layers.Dense(input_dim, activation='linear'),
            layers.Dense(input_dim*2, activation='linear'),
            layers.Dense(input_dim*4, activation='linear'),
            layers.Dense(input_dim*8, activation='linear'),
            layers.Dense(input_dim*4, activation='linear'),
            layers.Dense(input_dim*2, activation='linear'),
            layers.Dense(input_dim, activation='linear')
        ]

    def call(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
            x = self.activation(x)
            x = self.dropout(x)

        return self.classifier(x)

In [4]:
df = pd.read_csv(DB_PATH).drop(DROP_COLS, axis=1)
df

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
df.rename(RENAME_COLS, axis=1, inplace=True)
df

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,target
0,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
# encode categorical columns
for column in CAT_COLS:
    category = pd.Categorical(df[column])
    df[column] = category.codes
    print(column, ': ', category.view())

sex :  ['Male', 'Male', 'Male', 'Male', 'Female', ..., 'Female', 'Male', 'Male', 'Male', 'Male']
Length: 920
Categories (2, object): ['Female', 'Male']
cp :  ['typical angina', 'asymptomatic', 'asymptomatic', 'non-anginal', 'atypical angina', ..., 'asymptomatic', 'typical angina', 'asymptomatic', 'asymptomatic', 'atypical angina']
Length: 920
Categories (4, object): ['asymptomatic', 'atypical angina', 'non-anginal', 'typical angina']
restecg :  ['lv hypertrophy', 'lv hypertrophy', 'lv hypertrophy', 'normal', 'lv hypertrophy', ..., 'st-t abnormality', 'st-t abnormality', 'st-t abnormality', 'lv hypertrophy', 'lv hypertrophy']
Length: 920
Categories (3, object): ['lv hypertrophy', 'normal', 'st-t abnormality']
slope :  ['downsloping', 'flat', 'flat', 'downsloping', 'upsloping', ..., NaN, NaN, NaN, NaN, NaN]
Length: 920
Categories (3, object): ['downsloping', 'flat', 'upsloping']
thal :  ['fixed defect', 'normal', 'reversable defect', 'normal', 'normal', ..., NaN, NaN, 'fixed defect', NaN

In [7]:
X = df.loc[:, :'thal'].drop(columns='dataset')
y = df['target']

In [8]:
y_ohe = to_categorical(y)
y_ohe

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [9]:
X.describe()

,age,sex,cp,trestbps,chol,restecg,thalch,oldpeak,slope,ca,thal
count,920.000000,920.000000,920.000000,861.000000,890.000000,920.000000,865.000000,858.000000,920.000000,309.000000,920.000000
mean,53.510870,0.789130,0.782609,132.132404,199.130337,0.985870,137.545665,0.878788,0.480435,0.676375,0.102174
std,9.424685,0.408148,0.956350,19.066070,110.780810,0.638633,25.926276,1.091226,1.167966,0.935653,1.251939
min,28.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,60.000000,-2.600000,-1.000000,0.000000,-1.000000
25%,47.000000,1.000000,0.000000,120.000000,175.000000,1.000000,120.000000,0.000000,-1.000000,0.000000,-1.000000
50%,54.000000,1.000000,0.000000,130.000000,223.000000,1.000000,140.000000,0.500000,1.000000,0.000000,-1.000000
75%,60.000000,1.000000,2.000000,140.000000,268.000000,1.000000,157.000000,1.500000,1.000000,1.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,2.000000,202.000000,6.200000,2.000000,3.000000,2.000000


In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       920 non-null    int64  
 1   sex       920 non-null    int8   
 2   cp        920 non-null    int8   
 3   trestbps  861 non-null    float64
 4   chol      890 non-null    float64
 5   fbs       830 non-null    object 
 6   restecg   920 non-null    int8   
 7   thalch    865 non-null    float64
 8   exang     865 non-null    object 
 9   oldpeak   858 non-null    float64
 10  slope     920 non-null    int8   
 11  ca        309 non-null    float64
 12  thal      920 non-null    int8   
dtypes: float64(5), int64(1), int8(5), object(2)
memory usage: 62.1+ KB


In [11]:
X.isnull().sum()

age           0
sex           0
cp            0
trestbps     59
chol         30
fbs          90
restecg       0
thalch       55
exang        55
oldpeak      62
slope         0
ca          611
thal          0
dtype: int64

In [12]:
X.drop('ca', axis=1, inplace=True)

In [13]:
for column in X:
    if X[column].hasnans:
        X[column].fillna(X[column].mean(skipna=True), inplace=True)

In [14]:
scaler = MinMaxScaler()
X[SCALE_COLS] = scaler.fit_transform(X[SCALE_COLS])
X['age'] = X['age'] / 10

In [15]:
X = X.astype(DTYPE)
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,thal
0,6.3,1.0,3.0,0.725000,0.386401,1.0,0.0,0.633803,0.000000,2.300000,0.0,0.0
1,6.7,1.0,0.0,0.800000,0.474295,0.0,0.0,0.338028,1.000000,1.500000,1.0,1.0
2,6.7,1.0,0.0,0.600000,0.379768,0.0,0.0,0.485915,1.000000,2.600000,1.0,2.0
3,3.7,1.0,2.0,0.650000,0.414594,0.0,1.0,0.894366,0.000000,3.500000,0.0,1.0
4,4.1,0.0,1.0,0.650000,0.338308,0.0,0.0,0.788732,0.000000,1.400000,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
915,5.4,0.0,0.0,0.635000,0.552239,1.0,2.0,0.661972,0.000000,0.000000,-1.0,-1.0
916,6.2,1.0,3.0,0.660662,0.230514,0.0,2.0,0.546096,0.389595,0.878788,-1.0,-1.0
917,5.5,1.0,0.0,0.610000,0.369818,1.0,2.0,0.281690,0.000000,0.000000,-1.0,0.0
918,5.8,1.0,0.0,0.660662,0.638474,1.0,0.0,0.546096,0.389595,0.878788,-1.0,-1.0


In [16]:
X_train_, X_test, y_train_, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train_, y_train_, test_size=VALIDATION_SIZE, random_state=RANDOM_STATE)

In [17]:
y_train_ohe_ = to_categorical(y_train_)
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)
y_val_ohe = to_categorical(y_val)

In [18]:
optimizer = optimizers.Adam(LEARNING_RATE)
loss = losses.CategoricalCrossentropy()
mlp = MLP(input_dim=len(X.iloc[0]), output_dim=len(y_ohe[0]))
mlp.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
history = mlp.fit(X_train, y_train_ohe, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val_ohe))


Epoch 1/100


20/20 [==============================] - 3s 21ms/step - loss: 1.4392 - accuracy: 0.4016 - val_loss: 1.3935 - val_accuracy: 0.3964
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 1.3272 - accuracy: 0.4224 - val_loss: 1.3166 - val_accuracy: 0.4144
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 1.2995 - accuracy: 0.4448 - val_loss: 1.2651 - val_accuracy: 0.4324
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 1.2321 - accuracy: 0.5152 - val_loss: 1.2356 - val_accuracy: 0.5045
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 1.2231 - accuracy: 0.4816 - val_loss: 1.2230 - val_accuracy: 0.5045
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 1.2219 - accuracy: 0.5072 - val_loss: 1.2006 - val_accuracy: 0.5315
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 1.1743 - accuracy: 0.5440 - val_loss: 1.1879 - val_accuracy: 0.5225
Epoch 8/1

In [19]:
results = mlp.evaluate(X_test, y_test_ohe, batch_size=BATCH_SIZE)

6/6 [==============================] - 0s 3ms/step - loss: 0.9775 - accuracy: 0.6141


In [20]:
y_pred = np.argmax(mlp.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))

6/6 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.73      0.92      0.81        89
           1       0.57      0.46      0.51        56
           2       0.18      0.24      0.21        17
           3       0.50      0.06      0.10        18
           4       0.00      0.00      0.00         4

    accuracy                           0.61       184
   macro avg       0.39      0.34      0.33       184
weighted avg       0.59      0.61      0.58       184



In [21]:
gpc = GaussianProcessClassifier(kernel=RBF(), random_state=RANDOM_STATE)
gpc.fit(X_train_, y_train_)
gpc.score(X_test, y_test)

c:\Users\myself\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\myself\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\myself\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn

0.6304347826086957